<a href="https://colab.research.google.com/github/elemi10/7506-TP-Org-de-datos/blob/7506-Trabajo-Practico-1/TP1_7506_Org_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TP1 - Real or Not?**

###  **Análisis exploratorio**

![Banner_TP1.png](https://drive.google.com/uc?id=1BPA2RF1SDm9bTs1xZfVUa1VQn932E3wr)


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1RAGDjlzJ6spO5Sq8_x3UTIvxLhKAUBEt'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv') 

### Importamos **Librerias**

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime as dt
import scipy as sp
#from sklearn.cross_validation import train_test_split -- no levanta
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,9)
plt.style.use('default') # mejoramos esteticamente un poco los gráficos en matplotlib
sns.set(style='whitegrid') # seteando tipo de grid en seaborn

from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
#from sklearn.metrics import confusion_matrix
#from sklearn.naive_bayes import GaussianNB
#from sklearn.metrics import classification_report
#from sklearn.feature_selection import SelectKBest
#from sklearn.cross_validation import train_test_split --> No levanta el feature de la libreria

### Levantramos el archivo **train.csv**

In [0]:
df_train = pd.read_csv(r'train.csv', usecols=['keyword','location','text','target']) 
df_train.head()

## Preprocesamiento de datos

In [0]:
# Verifico que no haya instancias nulas o filas completas nulas
df_train=df_train.dropna(how="all")
df_train.info()

In [0]:
# Normalizo el dataframe los registros NaN con texto
df_train=df_train.fillna({'keyword': 'sin keyword',\
                   'location': 'sin location'})
df_train.info()

In [0]:
# Eliminamos los tweets duplicados considerando que su cantidad no tiene impacto en el análisis.
df_train=df_train.drop_duplicates('text')
df_train.info()

In [0]:
# Verificamos la existencia de valores nulos
df_train.isnull().any()

**→ Creamos una serie que hace el conteo de la cant de caracteres del tweet**

In [0]:
# Creo una columna con el cálculo de las palabras que tiene la columna 'text' y la llamo 'caracteres_tweet'. 
# Despues consulto con un head()
df_train['caracteres_tweet']=df_train['text'].str.len()
df_train.head(2)

### Cálculo de **estadísticas básicas**

In [0]:
# Calculamos unas estadisticas basicas
df_train.describe()

In [0]:
#Calculo el promedio y lo asigno a la variable 'df_short_mean'
df_train_mean=np.round(df_train['caracteres_tweet'].mean(), decimals=2)
df_train_mean

In [0]:
# Calculo la mediana, para tener presente como impacta max() y min() en la distribución
df_train_median=np.round(df_train['caracteres_tweet'].median(), decimals=2)
df_train_median

In [0]:
# cuento total de tweets analizados
total_tweets=len(df_train)
total_tweets

In [0]:
# verifico cuantos son verdaderos
tweets_verdaderos=(df_train['target']==1).sum()
tweets_verdaderos

In [0]:
# verifico cuantos son fakes
tweets_falsos=(df_train['target']==0).sum()
tweets_falsos

In [0]:
# calculo la probabilidad (casos favorables/sobre casos posibles) → en este casos es equivalente .mean()
probabilidad_tweet_verdadero = np.round((tweets_verdaderos)/(total_tweets),decimals=5)
probabilidad_tweet_verdadero

In [0]:
# calculo la probabilidad (casos favorables/sobre casos posibles) → en este casos es equivalente .mean()
probabilidad_tweet_falso = np.round((tweets_falsos)/(total_tweets),decimals=2)
probabilidad_tweet_falso

**→ Asignación de variables para cálculos**

In [0]:
# Asigno a variables para que se puedan operar alfabéticamente
tweets_verdaderos=df_train['target']==1
tweets_verdaderos.head()
# Verifico cantidad de tweets verdaderos
tweets_verdaderos.sum()

In [0]:
# Aplico la misma acción para tweets fakes
# verifico cantidad de tweets fakes
tweets_fake=df_train['target']==0
tweets_fake.head()
tweets_fake.sum()

In [0]:
# Asigno a la variable 'tweets_mayor_promedio', los tweets que tienen un promedio en cant de registros > Promedio (de caracteres de tweets del DF)
# Me va a servir para ver mas adelante como es su distribución
tweets_mayor_promedio=df_train['caracteres_tweet']>=(df_train_mean)
# Hago un .head() para verificar para despues hacer la suma de tweets con una cant de caracteres mayor al promedio
print(tweets_mayor_promedio.head())
tweets_mayor_promedio.sum()

In [0]:
# Lo mismo que el apartado anterior, con la diferencia que traigo en la variable 'tweets_menor_promedio' 
# aquellos registros que son menor al promedio
tweets_menor_promedio=df_train['caracteres_tweet']<(df_train_mean)
tweets_menor_promedio.sum()

In [0]:
# Por un tema de comodidad, creo una nueva columna con la referencia al valor booleano de la variable 'target'.
# 0=tweet falso | 1=tweet verdadero
df_train['tweet_type']=np.where(df_train['target']==1,"tweet verdadero","tweet falso")
df_train.head(2)

In [0]:
# Reverifico
pd.value_counts(df_train['tweet_type'])

In [0]:
# Armo una tabla de frecuencia de las variables
100*df_train['tweet_type'].value_counts()/len(df_train['tweet_type'])

In [0]:
# Agrego una categoría para agrupar por promedio
df_train['avg_category']=np.where(df_train['caracteres_tweet']>=(df_train_mean)\
                                  ,"tw w/characts.above AVG ocurrs.","tw w/characts.below AVG ocurrs") 
df_train.head()

In [0]:
# Relaciono las variables con una tabla de contingencia (%)
pd.crosstab(index=df_train['tweet_type'], columns=df_train['avg_category'], \
            margins=True).apply(lambda x:x/len(df_train)*100,axis=1)

### Gráfico **Overlaid Density Plots** [Distribución]

In [0]:
# grafico de tabla de contingencia de los tweets falsos vs tweets v en funcion del promedio de caracteres
graph_distrb=sns.kdeplot(df_train[df_train['tweet_type']=="tweet verdadero"]['caracteres_tweet'],\
                         color="orange", label="distribution on TRUE avg tweets", shade=True)

graph_distrb=sns.kdeplot(df_train[df_train['tweet_type']=="tweet falso"]['caracteres_tweet'], \
                         color="blue", label="distribution on FAKE avg tweets", shade=True)

graph_distrb.set_title("avg tweets characters Distribution", fontsize=18)
graph_distrb.set_ylabel("Densidad", fontsize=18)

### **Análisis & tokenización**

In [0]:
# Función de tokenización
import string
punctuation = set(string.punctuation)

def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

In [0]:
# Creo la serie 'text_vector'
df_train['text_vector']=df_train['text'].apply(tokenize)
df_train.head(2)

In [0]:
# Verifico integridad de DF
df_train.info()

In [0]:
# Descargo e importo lista de stopwords para filtrar
import nltk
nltk.download('stopwords')

In [0]:
from nltk.corpus import stopwords
stopwords.fileids()

In [0]:
# Asigno en la variable 'stop' la categoria de stopword por la que voy a realizar el filtro
stop=stopwords.words('english')

In [0]:
def filtro(text_vector):
  text_vector_filtrado = []
  for palabra in text_vector:
    if palabra not in stop:
      text_vector_filtrado.append(palabra)
  return(text_vector_filtrado)

In [0]:
# Tokens filtrados por stopwords en inglés
df_train['text_vector_filtrado']=df_train['text_vector'].apply(filtro)
df_train['text_vector_filtrado'].head()

In [0]:
# Asigno en una nueva columna el largo del vector 'text_vector'
df_train['elem_vector']=df_train['text_vector'].str.len()
df_train.head(2)

In [0]:
# Cuento los elementos del vector filtrado 'text_vector_filtrado'
df_train['elem_vector_filtrado']=df_train['text_vector_filtrado'].str.len()
df_train.head(2)

In [0]:
# Verifico estructura del DF
df_train.info()

### Preproceso y categorización de la variable ***Keyword***

In [0]:
# "Limpiamos" la serie de caracteres especiales
df_train['keyword']=df_train['keyword'].str.replace('%20','_')
# Verificamos actualización
df_train['keyword'].unique()

### Generación de **diccionario**

In [0]:
# Listo para generar un diccionario y traducir la serie 'keyword'
keyword_list=df_train['keyword'].unique().tolist()
print(keyword_list)

In [0]:
#1 Genero lista en espaniol
keyword_list_esp=['sin_keyword','en llamas','accidente','replica','accidente_avion','ambulancia','aniquilado','aniquilacion','apocalipse','armageddon','ejercito','incendio_intencional','piromano','ataque','atacado','avalancha','combate/batalla/lucha','bioterror','bioterrorismo','resplandecer/arder','resplandeciente/ardiente','hemorragia','exploto','plaga','ventisca','sangre','sangriento','estallido','bolsa_de_cadaver','embolsado_de_cadaver','bolsas_de_cadaver','bomba','bombardeado','bombardeo','colapso_de_puente','incendio_de_edificios','edificios_en_llamas','quemado','quemando/incendiando','incendio_edificios','incendio_de_matorrales/arbustos','bajas/muertes/perdidas','baja/muerte/perdida','catastrofe','catastrofico','emergencia_quimica','caida_acantilado','colapso','colapsado','colisionar','colisiono','colision','choque','estrellado','aplastamiento','aplastado','toque_de_queda','ciclon','da¤o','peligro','muerto','muerte','muertes','escombros','diluvio','inundado','demoler','demolido','demolicion','descarrilar','descarrilado','descarrilamiento','desolado','desolacion','destruir','destruido','destruccion','detonado','detonacion','devastado','devastacion','desastre','dislacado/desplazado','sequia','ahogar','ahogado','ahogamiento/ahogandose','tormenta_polvo','terremoto','electrocutar','electrocutado','emergencia','plan_de_emergencia','servicio_de_emergencia','engullido','epicentro','evacuar','evacuado','evacuacion','explotar','exploto/detonado/estallado','explosion','testigo_ocular','hambruna','fatal','fatalidades','fatalidades','miedo','fuego','camion_bombero','primeros_en_responder','llamas','aplastado','inundacion','inundaciones','inundaciones','incendio_forestal','incendio_forestales','granizo','tormenta_granizo','da¤o','peligro','peligroso','ola_de_calor','infierno','secuestro','secuestrador','secuestro','rehen','rehenes','huracan','herido/lesionado','heridas/lesiones','herida/lesion','inundado','inundacion','deslisamiento_tierra','lava','rayo','golpe_fuerte/estampido_fuerte','asesinato_en_serie','asesino_en_serie','masacre','caos','fusion','militar','alud_de_lodo','desastre_natural','desastre_nuclear','reactor_nuclear','obliterar/destruir/eliminar','borrado/destruido/eliminado','obliteracion/destruccion/eliminacion','derrame_de_petroleo','brote','pandemonio/confusion/caos','panico','tener_panico','polic¡a','cuarentena','en_cuarentena','emergencia_radiacion','tormenta','arrasado','refugiados','rescate','rescatado','rescatadores','disturbio','disturbios','escombros','ruina','tormenta_de_arena','grito','gritando','gritos','sismico','sumidero/pozo','hundiendose','sirena','sirenas','humo','tormenta_de_nieve','tormenta','camilla','fallo_estructural','bomba_suicida','suicida','bombardeo_suicida','hundido','sobrevivir','sobrevivio','sobreviviente','terrorismo','terrorista','amenaza','trueno','tormenta_electrica','tornado','tragedia','atrapado','trauma','traumatizado','problema','tsunami','tornado/torbellino','tifon','transtorno','tormenta_violenta','volcan','zona_de_guerra','arma','armas','torbellino','incendios_forestales','incendio_forestal','tormenta_de_viento','moretoneado','moretones','naufragio','restos','naufrago']
print(keyword_list_esp)

In [0]:
#1 Genero el diccionario eng-esp de la serie 'Keyword'
keyword_dicc={}
y=0
for x in keyword_list:
  keyword_dicc[x]=keyword_list_esp[y]
  y=y+1
  #print(keyword_dicc)

print(keyword_dicc)

In [0]:
# Mapeo los valores de las key del diccionario keyword_dicc
df_train['keyword_esp']=df_train['keyword'].map(keyword_dicc)
df_train.head(2)

In [0]:
# Verifico el top 20 de las categorias mas frecuentes para la serie keyword 
df_train['keyword_esp'].value_counts().sort_values(ascending=False).head(20)

In [0]:
# Hacemos un groupby de las categorias keyword con tweets verdaderos y falsos ('target')
df_agrupado=df_train.groupby(['keyword_esp','tweet_type']).size().sort_values(ascending=False)
df_agrupado

In [0]:
# Otro groupby de la categorias contando ocurrencias de tweets y de caracteres que los conforman
df_agrupado_ocurrencias=df_train.groupby(['keyword_esp']).agg({'target':['count'],'caracteres_tweet':['max','min','mean']})
df_agrupado_ocurrencias

### → **Scikit Learn**: Ejecución de **One Hot Encoding** para clasificar

In [0]:
# Dummifico las categorías
df_train_dummies=pd.get_dummies(df_train.keyword)
df_train_dummies.head(2)

In [0]:
# Hago un join con df_train y df_train_dummies
df_train_merged = pd.concat([df_train,df_train_dummies],axis='columns')
df_train_merged.head(2)

In [0]:
# Elimino las columnas 'keyword_esp' y 'sin keyword'
df_train_merged_final=df_train_merged.drop(['keyword_esp','sin keyword'],axis='columns')
df_train_merged_final.head(2)

In [0]:
# Exporto a .csv para verificar estructura
df_train_merged_final.to_csv('columna_texto_vector_filtrado.csv')

Vectorizo con el feature **CountVectorizer**

In [0]:
count_vector=CountVectorizer(
    tokenizer = tokenize,
    binary = True)

In [0]:
#count_vector.fit(documents)
#vector = count_vector.transform(documents)

In [0]:
#documents=df_train_merged_final['text_vector_filtrado']
#documents

In [0]:
#x_traincv=cv.fit_transform(df_train_merged_final[['text']])
#count_vector.fit(documents)

In [0]:
# Estoy teniendo problemas al querer pasar 'text_vector_filtrado' como raw document
# → Ver variable 'd'
#from sklearn.preprocessing import OneHotEncoder
#ohe=OneHotEncoder(sparse=False)

In [0]:
#ohe.fit_transform(df_train_merged_final['text_vector_filtrado'])

In [0]:
d=df_train_merged_final['text_vector_filtrado'].array
d

In [0]:
#documentos=d

In [0]:
#x_traincv=cv.fit_transform([d])
#count_vector.fit(d)

### → **Regresion Lineal**: Preproceso **dataframe 'X'** para clasificar

In [0]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()

In [0]:
# Dejo solamente el merge con las columnas que me interesan para correr un modelo del Regresion Lineal
X=df_train_merged_final.drop(df_train_merged_final.columns[[0,1,2,4,5,6,7,8]], axis='columns')
X.head(2)

In [0]:
# Exporto a .csv para verificar estructura
#X.to_excel('X_dummy.xlsx')
X.to_csv('X_dummy.csv')

In [0]:
y=df_train_merged_final.target
y

#### Verifico el modelo de regresión lineal con las variables X,y

In [0]:
LR_model.fit(X,y)

In [0]:
LR_model.predict([[1,0,0]]) #--> ver error en los parametros

In [0]:
# ver modelo de scoring
LR_model.score(X,y)